In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
import re
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import matplotlib.patches as mpatches
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import warnings
import optuna
from sklearn import svm
from tqdm.notebook import tqdm # for a continuous progress bar style
import random
#oversampling
warnings.filterwarnings('ignore')
%matplotlib inline


-GET PREPARE (data cleanup / curation, data source, privacy, preprocessing)
-EXPLORE (plots, patterns, correlations, outliers, anomalies)
-MODEL (train prediction, validation, tuning, scale and deployment)
-COMMUNICATE ([....])

feature engineering
feature extraction
feature selection : pearson correlation, wrapper method, lasso regression, explore with histogramms, scatter plots

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from itertools import compress
sfm_selector = SelectFromModel(estimator=LogisticRegression())
sfm_selector.fit(X, y)
list_of_selected_features = list(compress(feature_names, sfm_selector.get_support().tolist()))
print(list_of_selected_features)

In [ ]:
from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeRegressor
# define the model
model = LogisticRegression()
#model = DecisionTreeRegressor()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_
#importance = model.feature_importances_
#imp_features = [f for i_f, f in enumerate(feature_names) if abs(importance[0][i_f]) < 0.05]
importance = np.abs(importance[0])
feature_names_s = [feature_names[f] for f in importance.argsort()]
less_important_5 = feature_names_s[:5]
most_important_5 = feature_names_s[:-6:-1]
print("Less important features:")
print(less_important_5)
print("Most important features:")
print(most_important_5)

plot feature importance / ranking

INITIAL CLUSTERING

In [ ]:
# data reading
dataset = pd.read_csv("final_dataset.csv", index_col=0)

In [ ]:
dataset

Preprocessing

In [ ]:
dataset.head()

In [ ]:
model_data = dataset.values

DATA INSIGHTS - VISUALISATION

DATASET SPLIT

In [ ]:
#model_input, expected_output = model_data[:, -1], model_data[:, 1]

In [ ]:
# training and test data
x_train, x_test, y_train, y_test = None, None, None, None
#train_test_split(model_input, expected_output, test_size=0.20, random_state=1, shuffle=True)

In [ ]:
SCALING - STANDARIZATION[prefer] - NORMALIZATION(0,1)

In [ ]:
scaler = StandardScaler(with_mean=False) # no mean?

scaled_data_model = LogisticRegression(max_iter=1000)

pipe = make_pipeline(scaler, scaled_data_model)
pipe.fit(x_train, y_train)
Pipeline(steps=[('standardscaler', scaler),
                ('logisticregression', scaled_data_model)])

pipe.score(x_test,y_test)


method selection

types / number of features
dataset size
explainability
class priors
need for param tuning
incremental training
in/out memory training
prediction speed
train speed
performance
features indipendent

In [ ]:
# def evaluate methods (with or without scale)
#(create func for testing training and testing speed with plots)
models = []

lr = LogisticRegression(max_iter=1000)
models.append(("LogisticRegression",lr))
models.append(("KNeighbors",KNeighborsClassifier()))
models.append(("DecisionTree",DecisionTreeClassifier()))
svm.SVC()
models.append(("SVC",SVC()))
models.append(("Linear SVC",LinearSVC()))
models.append(("SVM",))
models.append(("NaiveBayes",GaussianNB()))
models.append(("RandomForest",RandomForestClassifier()))
models.append(("NeuralNetwork",MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500)))



results = []
names = []
for name,model in models:
    result = cross_val_score(model, x_train, y_train, cv=3, scoring = 'accuracy')
    names.append(name)
    results.append(result)
    

#for i in range(len(names)):
#    print(names[i],results[i].mean())

In [ ]:
means = []
for i in range(len(names)):
    means.append(results[i].mean())
plt.bar(names, means)
plt.title("Accurancy comparison")
plt.savefig('plot2.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
#model = 

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

space = dict()
# space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
# space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
# space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]

search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)
result = search.fit(x_train, y_train)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
clf = LogisticRegression(max_iter=1000)

#model train
clf.fit(x_train, y_train)

# prediction
prediction = clf.predict(x_test)

print("model score: ",clf.score(x_test, y_test))

In [ ]:
clf = LogisticRegression(penalty='l2', C=10, solver='newton-cg' ,max_iter=1000)

#model train
clf.fit(idf_train, output_train)
prediction = clf.predict(idf_validation)

# save the model
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))
  
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

#  *** INSERT TEST DATA HERE ***
result = loaded_model.score(idf_validation, output_validation)
print("loaded_model score: ",result)
print("--------------------------------")
#  *** --------------------- ***

print("\nModel Evaluation:")
print("accurancy: ",metrics.accuracy_score(output_validation,prediction))
print("precision: ",metrics.precision_score(output_validation,prediction,average='macro'))
print("recall: ",metrics.recall_score(output_validation,prediction,average='micro'))
print("f-measure: ",metrics.f1_score(output_validation,prediction,average='weighted'))

cm = confusion_matrix(output_validation,prediction)
 
plot_confusion_matrix(clf, idf_validation, output_validation)
plt.title('confusion matrix')
plt.savefig('plot3.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# below code is pulled from sklearn documentation

def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    scoring=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    scoring : str or callable, default=None
        A str (see model evaluation documentation) or
        a scorer callable object / function with signature
        ``scorer(estimator, X, y)``.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        scoring=scoring,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))


title = "Learning Curves for Logistic Regression (TFIDF)"


cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = LogisticRegression(penalty='l2', C=10, solver='newton-cg' ,max_iter=1000)
plot_learning_curve(
    estimator,
    title,
    idf_train,
    output_train,
    axes=axes[:, 0],
    ylim = (0.0,1.1),
    cv=cv,
    n_jobs=4,
    scoring="accuracy",
)


title = "Learning Curves for Logistic Regression (BOW)"


cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)

estimator = LogisticRegression(penalty='l2', C=10, solver='newton-cg' ,max_iter=1000)
plot_learning_curve(
    estimator,
    title,
    bow_train,
    output_train,
    axes=axes[:, 1],
    ylim = (0.0,1.1),
    cv=cv,
    n_jobs=4,
    scoring="accuracy",
)


plt.savefig('plot6.png', dpi=300, bbox_inches='tight')


plt.show()

overfitting underfitting (bias-variance) exploration with train and validaton error

cross validation

In [ ]:
# ***********MODEL-TESTING-INSERST-CSV-TEST-FILE-HERE***********

test = pd.read_csv('imdb-reviews.csv',sep = '\t', engine = 'python')  # insert test file here
test['review'] = test['review'].apply(remove_links)
test['review'] = test['review'].apply(lowercase)
test['review'] = test['review'].apply(remove_punctuation)
test['review'] = test['review'].apply(remove_stopwords)
test['review'] = test['review'].apply(remove_html)
test['rating'] = test['rating'].apply(sentiment_label)
test.drop(test[test['rating'] == None].index, inplace = True)

model_data_test = test.values
model_input_test, expected_output_test = model_data_test[:, -1], model_data_test[:, 1]
expected_output_test = expected_output_test.astype('str')


loaded_model = pickle.load(open('finalized_model.sav', 'rb'))
prediction2 = loaded_model.predict(idf_test2)
result = loaded_model.score(idf_test, expected_output_test)
print("model score:",result)

print("\nModel Evaluation:")
print("accurancy: ",metrics.accuracy_score(expected_output_test,prediction2))
print("precision: ",metrics.precision_score(expected_output_test,prediction2,average='macro'))
print("recall: ",metrics.recall_score(expected_output_test,prediction2,average='micro'))
print("f-measure: ",metrics.f1_score(expected_output_test,prediction2,average='weighted'))

# **************************************************************

pr curve

dimentionality reduction

In [ ]:
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [ ]:
print(classification_report(test_y, preds))

In [ ]:
# search space
def objective(trial):
    
    params = {
              'learning_rate': trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3]),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_unit': trial.suggest_categorical("n_unit", [64, 128]),
              'epochs': trial.suggest_categorical("epochs", [20, 50, 80, 100]),
              'batch_size': trial.suggest_categorical("batch_size", [16, 32, 64]),
              'loss_funct': trial.suggest_categorical("loss_funct", ["MSELoss", "BCELoss"]),
              'dropout': trial.suggest_categorical("dropout", [True, False]),
              'activationFunction': trial.suggest_categorical("activationFunction", ["LeakyReLU", "ReLU"])
             }
    
    model = build_model(params,trial)
    
    accuracy = train_and_evaluate(params,model,trial)
    
    return accuracy

In [ ]:
def build_model(params,trial):
    
    # number of layers in network
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    # Define layer sizes
    in_features = x_train.shape[1]
    #in_features = 200
    nodes =  params['n_unit']
    out_features = nodes
    D_out = 1
    # flag to add dropout layer
    withDropout = params['dropout']
    activationFunction = getattr(nn, params['activationFunction'])() 

    for i in range(n_layers):
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(activationFunction)
        if(withDropout):
            layers.append(nn.Dropout(p=0.5))
        
        
        # next layer nodes ar half of previous layer
        in_features = out_features
        out_features = out_features // 2  
        
    # output layer
    layers.append(nn.Linear(in_features,D_out))
    layers.append(nn.Sigmoid())       

    #Initialise model
    
    return nn.Sequential(*layers)

In [ ]:
def build_model(params,trial):
    
    # number of layers in network
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    # Define layer sizes
    in_features = x_train.shape[1]
    #in_features = 200
    nodes =  params['n_unit']
    out_features = nodes
    D_out = 1
    # flag to add dropout layer
    withDropout = params['dropout']
    activationFunction = getattr(nn, params['activationFunction'])() 

    for i in range(n_layers):
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(activationFunction)
        if(withDropout):
            layers.append(nn.Dropout(p=0.5))
        
        
        # next layer nodes ar half of previous layer
        in_features = out_features
        out_features = out_features // 2  
        
    # output layer
    layers.append(nn.Linear(in_features,D_out))
    layers.append(nn.Sigmoid())       

    #Initialise model
    
    return nn.Sequential(*layers)

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

In [ ]:
print("--------------------------------------------")
print("Model confusion matrix: \n",confusion_matrix(y_validation, y_pred_list))
print("--------------------------------------------")
print("Model classification report: \n",classification_report(y_validation, y_pred_list))
print("--------------------------------------------")
print("Model accuracy: ",metrics.accuracy_score(y_validation, y_pred_list))
print("--------------------------------------------")

plt.plot(epoch_acc_train,'-o')
plt.plot(epoch_acc_val,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train', 'Validation'])
plt.title('Train vs Validation Accuracy')
plt.show()

plt.plot(epoch_loss_train,'-o')
plt.plot(epoch_loss_val,'-o')
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train', 'Validation'])
plt.title('Train vs Validation Losses')
plt.show()

false_positive_rate, true_positive_rate, threshold = roc_curve(y_validation, y_probab_list)
print("--------------------------------------------")
print('roc_auc_score for model: ', roc_auc_score(y_validation, y_probab_list))
print("--------------------------------------------")
plt.title('Receiver Operating Characteristic - Neural Network')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
def model_test(model,testloader, y_test):
    
    y_test = y_test.squeeze().tolist()
    
    y_pred_list = []
    y_probab_list = []
    model.eval()
    with torch.no_grad():
        for X_batch, y_label in testloader:
            y_test_pred = model(X_batch)
            y_pred_tag = torch.round(y_test_pred)
            y_pred_list.append(y_pred_tag.numpy())
            y_probab_list.append(y_test_pred.numpy())

    y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
    y_probab_list = [a.squeeze().tolist() for a in y_probab_list]
    
        
    print("--------------------------------------------")
    print("Model confusion matrix: \n",confusion_matrix(y_test, y_pred_list))
    print("--------------------------------------------")
    print("Model classification report: \n",classification_report(y_test, y_pred_list))
    print("--------------------------------------------")
    print("Model accuracy: ",metrics.accuracy_score(y_test, y_pred_list))
    print("--------------------------------------------")

    
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_probab_list)
    print("--------------------------------------------")
    print('roc_auc_score for model: ', roc_auc_score(y_test, y_probab_list))
    print("--------------------------------------------")
    plt.title('Receiver Operating Characteristic - Neural Network')
    plt.plot(false_positive_rate, true_positive_rate)
    plt.plot([0, 1], ls="--")
    plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
SEED = None
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
plt.style.use('seaborn') # use seaborn style plotting

#file_paths

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
plot_optimization_history(study)
plot_parallel_coordinate(study)
optuna.visualization.plot_param_importances(study)
optuna.visualization.plot_contour(study, params=['num_layers', 'hidden_size'])
optuna.visualization.plot_contour(study, params=['cell_type', 'num_layers'])